In [127]:
import numpy as np 
import pandas as pd
import requests
from groq import Groq
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import math
from openai import OpenAI
import time

In [128]:
contracts = pd.read_csv("/home/xikron/Projects/misc/data/blacktesting/c17.csv")
contracts['Start Date']= pd.to_datetime(contracts['Start Date'])
contracts=contracts.drop_duplicates(subset='Award ID').sort_values(by='Start Date')
contracts.head()

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
0,36094308,VA24017E0199,MCK,6.619690e+07,NaN,EXPRESS REPORT: PHARMACY PRIME VENDOR (PPV) FY...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Veterans Affairs,Department of Veterans Affairs,2017-01-01,2017-03-31,b54de748-ee64-a1e6-546e-e7007b054797-C,NaN,561.0,department-of-veterans-affairs,CONT_AWD_VA24017E0199_3600_VA797P12D0001_3600
12,37217793,VA77017E0793,MCK,2.346173e+08,NaN,EXPRESS REPORT: PHARMACY PRIME VENDOR CMOP FY1...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Veterans Affairs,Department of Veterans Affairs,2017-01-01,2017-01-31,b54de748-ee64-a1e6-546e-e7007b054797-C,NaN,561.0,department-of-veterans-affairs,CONT_AWD_VA77017E0793_3600_VA797P12D0001_3600
11,23783261,HSBP1017J00044,SAIC,3.426015e+07,NaN,"IGF::CL,CT::IGF THIS TASK ORDER IS ISSUED TO D...",DELIVERY ORDER,['Q'],NaN,NaN,...,NaN,Department of Homeland Security,U.S. Customs and Border Protection,2017-01-01,2019-08-31,900dcfe5-fb04-162f-2149-75b51d42021a-C,NaN,766.0,department-of-homeland-security,CONT_AWD_HSBP1017J00044_7014_HSBP1013D00021_7014
10,14958476,FA811017F0001,NOC,1.202825e+08,NaN,IGF::OT::IGF CY17 B-2 PROGRAMMED DEPOT MAINT...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2017-01-01,2020-09-30,a4e18d9e-ec75-2c26-9475-543f727fdbe7-C,NaN,1173.0,department-of-defense,CONT_AWD_FA811017F0001_9700_FA861614D6060_9700
8,14958478,FA811017F0003,NOC,4.265573e+07,NaN,IGF::OT::IGF CY17 PERFORMANCE-BASED LOGISTICS...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2017-01-01,2017-12-31,a4e18d9e-ec75-2c26-9475-543f727fdbe7-C,NaN,1173.0,department-of-defense,CONT_AWD_FA811017F0003_9700_FA861614D6060_9700


In [129]:
contracts.shape

(645, 21)

In [131]:
def setLinkIntd (ticker, start, end, freq):
    link="https://financialmodelingprep.com/api/v3/historical-chart/"+freq+"/"+ticker+"?from="+start+"&to="+end+"&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"
    return link

def setLinkEod (ticker, start, end):
    return"https://financialmodelingprep.com/api/v3/historical-price-full/"+ticker+"?from="+start+"&to="+end+"&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"

In [132]:
response = requests.get(url=setLinkIntd("LMT", "2010-03-31", "2010-03-31", "5min"))
response1 = requests.get(url=setLinkEod("RTX", "2001-01-01", "2001-01-02"))

In [133]:
response1.json()['historical']
prices = pd.Series([entry['close'] for entry in response1.json()['historical']])        
print(prices)

0    22.19
dtype: float64


In [134]:
from dataclasses import dataclass

@dataclass
class transaction:
    contractID: int
    num: int

In [135]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

def getBuyDay(contractDate):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    buyDay = contractDate - 3 * uBday
    return buyDay.strftime('%Y-%m-%d')


In [136]:
def meanReversion(prices, window=20, threshold=2.5):
    if len(prices) < window:
        print(f"Warning: Not enough data points for rolling window (got {len(prices)}, expected {window})")
        return False  

    sma = prices.rolling(window=window).mean()
    std_dev = prices.rolling(window=window).std()

    z_score = (prices - sma) / std_dev

    if len(z_score) > 0:
        if z_score.iloc[-1] > threshold:
            return True  
        elif z_score.iloc[-1] < -threshold:
            return True
    else:
        print("Error: 'z_score' is empty or invalid.")
    
    return False  

In [137]:
def getSellDay(contractDate, ticker):
    us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    
    contractDate = pd.Timestamp(contractDate)
    adjDate = contractDate if us_business_day.is_on_offset(contractDate) else contractDate + us_business_day
    adjDate += 3 * us_business_day  
    
    startDay = adjDate - pd.Timedelta(days=30)
    endDay = adjDate
    
    response = requests.get(setLinkIntd(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d')), "5min"))
    
    if response.status_code == 200:
        data = response.json()
        if len(data)==0: 
            try:
                response = requests.get(setLinkEod(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d'))))
                data = response.json()['historical']
            except Exception as e: 
                print("Unexpected exception: "+ str(e))
                return None
        if len(data)!= 0:
            prices = pd.Series([entry['close'] for entry in data])        
            
            if meanReversion(prices):
                return adjDate.strftime('%Y-%m-%d')  
            else:
                next_day = adjDate + us_business_day
                return getSellDay(next_day.strftime('%Y-%m-%d'), ticker)  
        else: 
            return
    else:
        print(f"Error fetching data: {response.status_code}. Returning default sell day")
        return adjDate 


In [138]:
def buy(ticker, valuation, buyDate, id): 
    global portfolio, capital
    response = requests.get(url=setLinkIntd(ticker, buyDate, buyDate, "5min"))

    if response.status_code == 200: 
        if len(response.json()) == 0: 
            response = requests.get(url=setLinkEod(ticker, buyDate, buyDate))
            try: 
                pps = response.json()['historical']['close']
            except Exception as e: 
                print("Unexpected exception: "+ str(e))
                return False
        if len(response.json())>15:
            pps = response.json()[-15]["open"]

        if (pps!=None): 
            shares = math.floor(valuation/pps)
            if ticker not in portfolio: 
                portfolio[ticker]= [transaction(contractID=id, num=shares)] 
            else: 
                portfolio[ticker].append(transaction(contractID=id, num=shares))
            capital-=shares*pps 
            print("purchased "+ str(shares) + " shares of "+ticker+" at "+ str(pps)+" on "+buyDate+". current portfolio valued at "+ str(capital))
            return True 
        else: 
            return False
    else:
        return False 

            

In [141]:
def sell(ticker, sellDate, id):
    global portfolio, capital, valuation
    response = requests.get(url=setLinkIntd(ticker, sellDate, sellDate, "5min"))
    if response.status_code == 200 and len(response.json()) >15: 
        pps = response.json()[-15]["open"]
        for t in portfolio[ticker]: 
            if t.contractID == id: 
                shares = t.num
                value = shares*pps
                capital+=value 
                portfolio[ticker].remove(t)
                valuation.append(capital)
                print("sold "+str(shares) +" shares of " + ticker + " at " + str(pps)+" on "+sellDate+". current portfolio is valued at " + str(capital))
                return True 

In [143]:
capital = 10000000
portfolio = {}
valuation = []

In [144]:
global capital
capital = 100000000
for i in range(len(contracts)): 
    if (buy(contracts.iloc[i]['Recipient Name'], capital*0.15, getBuyDay(contracts.iloc[i]['Start Date']), contracts.iloc[i]['internal_id'])):
        sell(contracts.iloc[i]['Recipient Name'], getSellDay(contracts.iloc[i]['Start Date'], ticker=contracts.iloc[i]['Recipient Name']), contracts.iloc[i]['internal_id'])
    else:
        continue

purchased 106007 shares of MCK at 141.5 on 2016-12-28. current portfolio valued at 85000009.5
sold 106007 shares of MCK at 147.82 on 2017-02-16. current portfolio is valued at 100669964.24
purchased 106717 shares of MCK at 141.5 on 2016-12-28. current portfolio valued at 85569508.74
sold 106717 shares of MCK at 147.82 on 2017-02-16. current portfolio is valued at 101344415.67999999
purchased 179391 shares of SAIC at 84.74 on 2016-12-28. current portfolio valued at 86142822.33999999
sold 179391 shares of SAIC at 82.75 on 2017-02-06. current portfolio is valued at 100987427.58999999
purchased 65324 shares of NOC at 231.89 on 2016-12-28. current portfolio valued at 85839445.22999999
sold 65324 shares of NOC at 282.766 on 2017-09-22. current portfolio is valued at 104310851.41399999
purchased 67474 shares of NOC at 231.89 on 2016-12-28. current portfolio valued at 88664305.55399999
sold 67474 shares of NOC at 282.766 on 2017-09-22. current portfolio is valued at 107743658.638
purchased 649

KeyboardInterrupt: 

In [122]:
print(valuation)

[100874557.75, 101756763.25, 101398322.46000001, 102361839.56000002, 103334508.53000003, 104508325.89000003, 104597623.15000004, 105591533.86000003, 105681756.72000003, 106606004.22000003, 106890764.22000003, 107729872.38000003, 107589889.56500003, 107681819.44500002, 108705036.70500001, 108484262.65500002, 109099420.90500002, 108936261.40500002, 108217355.62500001, 107308519.78500001, 107670233.885, 108352128.81500001, 109078442.84300002, 109751528.43300003, 109718008.26300003, 109251918.39300002, 110152143.63300002, 110713049.46300001, 111002208.08800001, 111676246.05800001, 111201836.90050001, 111035034.20050001, 111736262.5205, 113292588.7205, 113257460.6005, 113732285.8205, 114759437.3405, 115733433.8605, 116262574.16049999, 117167915.9605, 117639548.9905, 118113083.48050001, 118664397.56550002, 119863497.77550003, 120044225.30550003, 120283463.46550003, 120507592.24550003, 120419356.84550004, 120786269.67440005, 121082777.19440004, 122093464.23440003, 121648060.03440003, 12120427